# Text Classifier

Chapter 5 of Programming PyTorch for Deep Learning, but using samples from the [TREC 2005 Spam Corpus](https://trec.nist.gov/data/spam.html) instead of tweets.

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data 
import torchtext

In [2]:
def my_tokenize(s):
    return s.split()

In [3]:
TEXT = data.Field(lower=True, tokenize=my_tokenize)
LABEL = data.Field(lower=True)
samples = data.TabularDataset(path='./data/ham-spam-samples.tsv',
                              format='tsv', 
                              fields=[("label",LABEL), ("statement",TEXT)],
                              skip_header=False)

In [4]:
(training, testing, validating) = samples.split(split_ratio=[0.6,0.2,0.2])
(len(training),len(testing),len(validating))

(181, 60, 60)

In [5]:
vocab_size = 30000
TEXT.build_vocab(training, max_size = vocab_size)
LABEL.build_vocab(training)
TEXT.vocab.freqs.most_common(10)

[('the', 1569),
 ('to', 1027),
 ('of', 847),
 ('and', 820),
 ('>', 618),
 ('in', 528),
 ('a', 459),
 ('this', 439),
 ('for', 387),
 ('is', 379)]

In [6]:
len(TEXT.vocab)

7607

In [7]:
vars(training.examples[0])

{'label': ['ham'],
 'statement': ['per',
  'eric',
  'moon',
  '-',
  'attached',
  'you',
  'will',
  'find',
  'the',
  'structuring',
  'slide',
  'for',
  'citibank',
  'prepay,',
  'with',
  'the',
  'changes.',
  'melissa',
  'solis',
  'enron',
  'north',
  'america',
  'melissa.solis@enron.com',
  'w:',
  '713-853-5167',
  'f:',
  '713-646-3460']}

In [8]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [9]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
(training, validating, testing), 
batch_size = 32,
device = device,
sort_key = lambda x: len(x.statement),
sort_within_batch = False)

# Defining the model

Start with a simple [Long short-term memory (LSTM)](https://en.wikipedia.org/wiki/Long_short-term_memory) model. 

Unlike the book, which relies on a three-part classifier, this model is doing a binary comparison, with an activation ([sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function)) function.

In [10]:
class BasicLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(BasicLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1)
        self.predictor = nn.Linear(hidden_size, 1)
        self.activator = nn.Sigmoid()

    def forward(self, seq):
        output, (hidden, _) = self.encoder(self.embedding(seq))
        prediction = self.predictor(torch.squeeze(hidden))
        prediction = self.activator(prediction)
        return prediction

In [11]:
training_example = next(iter(train_iterator))
vars(training_example)

{'batch_size': 32,
 'dataset': <torchtext.data.dataset.Dataset at 0x7fd466925ad0>,
 'fields': dict_keys(['label', 'statement']),
 'input_fields': ['label', 'statement'],
 'target_fields': [],
 'label': tensor([[2, 2, 3, 2, 2, 3, 3, 2, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 3, 2, 2, 2, 2, 2, 2]]),
 'statement': tensor([[5070,   30, 1193,  ...,  714,  910,  316],
         [  13, 1119,  538,  ...,  714,   27, 2768],
         [ 138,  137,  606,  ...,  143,   13, 6334],
         ...,
         [   1,    1,  375,  ...,    1,    1,    1],
         [   1,    1,  541,  ...,    1,    1,    1],
         [   1,    1,  580,  ...,    1,    1,    1]])}

In [12]:
model = BasicLSTM(100, 300, 30000)
model.to(device)

BasicLSTM(
  (embedding): Embedding(30000, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=1, bias=True)
  (activator): Sigmoid()
)

In [13]:
def train(epochs, nn, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs + 1):

        training_loss = 0.0
        valid_loss = 0.0
        nn.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = nn(batch.statement)
            loss = criterion(predict, batch.label.float())
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.statement.size(0)
        training_loss /= len(train_iterator)
 
        
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.statement)
            loss = criterion(predict, batch.label.float())
            valid_loss += loss.data.item() * batch.statement.size(0)
 
        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

In [14]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [15]:
train(5, model, optimizer, criterion, train_iterator, valid_iterator)

/home/denis/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1, 32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/home/denis/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1, 21])) that is different to the input size (torch.Size([21, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/home/denis/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1, 28])) that is different to the input size (torch.Size([28, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduct

Epoch: 1, Training Loss: -1362.23, Validation Loss: -2794.45
Epoch: 2, Training Loss: -4558.47, Validation Loss: -4524.70
Epoch: 3, Training Loss: -6600.54, Validation Loss: -5774.53
Epoch: 4, Training Loss: -8401.95, Validation Loss: -6795.56
Epoch: 5, Training Loss: -9089.07, Validation Loss: -7634.39


# Making Predictions

In [16]:
def classify_text(text):
    categories = {0: 'ham', 1: 'spam'}
    processed = TEXT.process([TEXT.preprocess(text)])
    processed = processed.to(device)
    return categories[model(processed).argmax().item()]

In [17]:
classify_text(testing.examples[0].statement)

'ham'

In [18]:
testing.examples[0].label[0]

'ham'

In [19]:
correct = 0
examined = 0
for test_example in testing.examples:
    actual = test_example.label[0]
    predicted = classify_text(test_example.statement)
    examined += 1
    if actual == predicted:
        correct += 1
        print('Correct   --> {}/{} right overall {:.2%}'.format(correct, examined, correct / examined))
    else:
        print('Incorrect --> {}/{} right overall {:.2%}'.format(correct, examined, correct / examined))

Correct   --> 1/1 right overall 100.00%
Correct   --> 2/2 right overall 100.00%
Correct   --> 3/3 right overall 100.00%
Incorrect --> 3/4 right overall 75.00%
Correct   --> 4/5 right overall 80.00%
Correct   --> 5/6 right overall 83.33%
Incorrect --> 5/7 right overall 71.43%
Incorrect --> 5/8 right overall 62.50%
Incorrect --> 5/9 right overall 55.56%
Correct   --> 6/10 right overall 60.00%
Correct   --> 7/11 right overall 63.64%
Incorrect --> 7/12 right overall 58.33%
Correct   --> 8/13 right overall 61.54%
Incorrect --> 8/14 right overall 57.14%
Correct   --> 9/15 right overall 60.00%
Incorrect --> 9/16 right overall 56.25%
Correct   --> 10/17 right overall 58.82%
Incorrect --> 10/18 right overall 55.56%
Correct   --> 11/19 right overall 57.89%
Incorrect --> 11/20 right overall 55.00%
Incorrect --> 11/21 right overall 52.38%
Correct   --> 12/22 right overall 54.55%
Correct   --> 13/23 right overall 56.52%
Correct   --> 14/24 right overall 58.33%
Correct   --> 15/25 right overall 60.0